In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import os
import joblib

import math 
import pandas as pd
import numpy as np
import datetime as dt
from pm4py.objects.log.util import dataframe_utils

from tqdm.auto import tqdm
import time

from sklearn.ensemble import RandomForestRegressor


In [3]:
os.listdir()

['.ipynb_checkpoints',
 '01_Get_config-done_checked.ipynb',
 '02_Comp_load-combined-done-checked.ipynb',
 'configs',
 'dur_range_dic.pickle',
 'evlog.csv',
 'get_dur_range_dic.ipynb',
 'get_load_locations_done.ipynb',
 'load_locations.pickle',
 'old_scripts']

In [4]:
#probably no need to include this - just illustrating how the framework works

def get_train_test(log, train_path, valid_path, index):
    
    data_train = pd.read_csv(train_path, index_col=index)
    data_valid = pd.read_csv(valid_path, index_col=index)
    
    evlog_train = log[log.case_id.isin(data_train.append(data_valid).index)]
    
    return evlog_train


#def get_train_test(log):
    
#    data_train = pd.read_csv('D:/Users/u0126338/Desktop/IC-feat/Exp_291221/BPIC12/data/data_train_ind.csv', index_col = 'CaseID')
#    data_valid = pd.read_csv('D:/Users/u0126338/Desktop/IC-feat/Exp_291221/BPIC12/data/data_valid_ind.csv', index_col = 'CaseID')

#    evlog_train = log[log.case_id.isin(data_train.append(data_valid).index)]
    
#    return evlog_train

In [5]:

def get_rf_relation(target_log, load, depth = 3,  threshold = 50):
    remtimes, loads = target_log.align(load, join='inner', copy=False)
    
    if len(loads) > threshold:
        lr = RandomForestRegressor(max_depth=depth).fit(np.array(loads).reshape(-1,1), remtimes)
        r2 = lr.score(np.array(loads).reshape(-1,1), remtimes)
    
    else:
        r2 = np.nan

    return r2


def get_location_config(log, location, threshold = 50):
    
    """Takes as input an event log, name of location of interest and the range of candidate durations for that location
    returns an optimal duration for that location. Can run seperately for all locations of interest"""
    
    all_activities = log.activity.value_counts().index
    dur_range_dic = joblib.load('dur_range_dic.pickle')
    
    duration_range = dur_range_dic[location]
    
    config = {}
        
    for target_activity in tqdm(all_activities):
            
        if (target_activity == '<EOS>'):
            config[target_activity] = (np.nan, 0, np.nan)
            continue
        
        target_log = log.loc[log.activity == target_activity, ].remtime
        
        if len(target_log) <= threshold:
            config[load_location] = (np.nan, 0, duration_range[0])    
        else:
            offset = 0
            best = None

            for diff in duration_range:
                load = log.loc[log.activity == location, ]\
                        .groupby(['ts']).count().asfreq('1S')\
                        .shift(1, freq=pd.DateOffset(hours=offset))\
                        .rolling(f"{diff}min").count()['ts_col']

                relation = get_rf_relation(target_log, load, depth = 3)
                if best is None or relation > best[0]:
                    best = (relation, offset, diff)

            config[target_activity] = best

    return config
    

def get_configurations(log):
    
    locations = joblib.load('load_locations.pickle')
    
    configurations = {}
    for location in tqdm(locations):
        print(location)
        
        configurations[location] = get_location_config(log, location)
            
    return configurations


In [6]:
log_csv = pd.read_csv('evlog.csv', sep=',')
log_csv.ts = log_csv.ts.apply(lambda x: x[:-4])
log_csv.drop(log_csv.columns[0], axis=1, inplace=True)
log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
log_csv  = get_train_test(log_csv, 
                          train_path='D:/Users/u0126338/Desktop/IC-feat/Exp_291221/BPIC12/data/data_train_ind.csv',
                          valid_path='D:/Users/u0126338/Desktop/IC-feat/Exp_291221/BPIC12/data/data_valid_ind.csv',
                          index='CaseID')

log_csv = log_csv.sort_values('ts')
log_csv.set_index(log_csv.ts, inplace=True)
log_csv = log_csv.rename(columns={'ts':'ts_col'})

In [7]:
#log_csv = log_csv.sample(frac=0.1)

In [8]:
configurations = get_configurations(log_csv)

W_Completeren aanvraag


NameError: name 'load_location' is not defined

In [ ]:
configurations['W_Completeren aanvraag']

In [ ]:
#compare obtained configs to the once previously obtained - should be pretty much the same

In [16]:
def get_all_configs(conf_dir = 'D:/Users/u0126338/Desktop/IC-feat/Exp_230622_gc/BPIC12/opt_dur/scripts/01_get_config/configs/'):
        
    configurations = {}

    for conf in os.listdir(conf_dir):

        if 'configuration' in conf:
            configurations.update(joblib.load(conf_dir + conf))

    print('config keys:', list(configurations.keys()))
    
    return configurations

configurations_old = get_all_configs()

config keys: ['W_Afhandelen leads', 'W_Completeren aanvraag', 'W_Nabellen incomplete dossiers', 'W_Nabellen offertes', 'W_Valideren aanvraag']


In [17]:
configurations_old['W_Completeren aanvraag']

{'W_Completeren aanvraag': (0.027214634563959006, 0, 1155),
 'W_Nabellen offertes': (0.05266604907878969, 0, 315),
 'W_Nabellen incomplete dossiers': (0.017465628853784554, 0, 1155),
 'W_Valideren aanvraag': (0.018471341689597187, 0, 945),
 'W_Afhandelen leads': (0.013920642308963949, 0, 645),
 '<BOS>': (0.02042672425192049, 0, 1125),
 'A_PARTLYSUBMITTED': (0.02045013028479037, 0, 1125),
 '<EOS>': (1.0, 0, 15),
 'A_SUBMITTED': (0.020721609853783796, 0, 1125),
 'A_DECLINED': (0.04911199067722083, 0, 555),
 'A_PREACCEPTED': (0.02826687338089673, 0, 1095),
 'O_SENT': (0.03907995802519071, 0, 1155),
 'O_CREATED': (0.039099115112104066, 0, 1155),
 'O_SELECTED': (0.03884322830124154, 0, 1155),
 'A_ACCEPTED': (0.030246094594274076, 0, 1125),
 'A_FINALIZED': (0.031369569779323814, 0, 1125),
 'O_CANCELLED': (0.11674144220529203, 0, 255),
 'O_SENT_BACK': (0.0403918567461361, 0, 1065),
 'A_CANCELLED': (0.13389714377732187, 0, 1005),
 'A_REGISTERED': (0.42124633561210656, 0, 885),
 'A_ACTIVATED': 